<a href="https://colab.research.google.com/github/wangbxj1234/voxel/blob/main/multi_scale_voxelize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import torch

In [78]:
def collate_fn(coord, feat, label):#(B,N,C)
    #coord, feat, label = list(zip(*batch))#(B,N,C)
    offset = [64,128]
    
    ####
    #coord, feat, label = torch.cat(coord,dim=0), torch.cat(feat,dim=0), torch.cat(label,dim=0) #但是属于不同batch的，就算坐标近也不能分进一个v。
    offset = torch.IntTensor(offset)##(N,3),(N,C),(N,),O

    ##to np
    coord, feat, label = coord.cpu().detach().numpy(),feat.cpu().detach().numpy(),label.cpu().detach().numpy()
    ##
    PN=0;
    voxel=[]
    voxel_1=[]
    voxel_2=[]
    voxel_3=[]
    for o in offset:
        c, f, l =coord[PN:int(o),:],feat[PN:int(o),:],label[PN:int(o),:]#(n,C),(n,C),(n,)
        discrete_coord = np.floor(c *4) 
        assert discrete_coord.ndim == 2
    # Floor first for negative coordinates
        arr = discrete_coord.copy()
        arr = arr.astype(np.uint64, copy=False)
        hashed_arr = np.uint64(14695981039346656037) * np.ones(arr.shape[0], dtype=np.uint64)
        for j in range(arr.shape[1]):
            hashed_arr *= np.uint64(1099511628211)
            hashed_arr = np.bitwise_xor(hashed_arr, arr[:, j])
        idx_sort = np.argsort(hashed_arr) 
        key_sort = hashed_arr[idx_sort] 
        _, count = np.unique(key_sort, return_counts=True)
        ### multiscale

###4    
        b = [count[i:i+4] for i in range(0,len(count),4)]
        count1=[]
        for i in range(len(b)):
            count1.append(np.sum(b[i]))
###16
        b = [count[i:i+16] for i in range(0,len(count),16)]
        count2=[]
        for i in range(len(b)):
            count2.append(np.sum(b[i]))
###32   
        b = [count[i:i+32] for i in range(0,len(count),32)]
        count3=[]
        for i in range(len(b)):
            count3.append(np.sum(b[i]))
        ###
        coord[PN:int(o),:],feat[PN:int(o),:],label[PN:int(o),:]= c[idx_sort],f[idx_sort],l[idx_sort]
        voxel.append(count)
        voxel_1.append(count1)
        voxel_2.append(count2)
        voxel_3.append(count3)
        PN=int(o)

    ##to torch
    coord, feat, label = torch.tensor(coord),torch.tensor(feat),torch.tensor(label)
    ##    

    return coord, feat, label, offset, voxel, voxel_1, voxel_2, voxel_3

In [79]:
#coord, feat, label=torch.rand(2,6,3),torch.rand(2,6,4),torch.rand(2,6,1)
coord, feat, label=torch.rand(2*64,3),torch.rand(2*64,32),torch.rand(2*64,1)
coord[0,:]=coord[1,:]
#coord

In [84]:
coord, feat, label, offset, voxel, voxel_1, voxel_2, voxel_3 = collate_fn(coord, feat, label)
print(coord.size(),feat.size(),label.size())

torch.Size([128, 3]) torch.Size([128, 32]) torch.Size([128, 1])


In [85]:
voxel_1

[[6, 5, 7, 8, 6, 10, 8, 7, 7], [5, 9, 5, 6, 6, 5, 8, 6, 8, 5, 1]]

In [86]:
voxel_2

[[26, 31, 7], [25, 25, 14]]

In [87]:
voxel_3

[[57, 7], [50, 14]]

In [88]:
v=torch.IntTensor(np.cumsum(np.concatenate(voxel, axis=0)))
v1=torch.IntTensor(np.cumsum(np.concatenate(voxel_1, axis=0)))
v2=torch.IntTensor(np.cumsum(np.concatenate(voxel_2, axis=0)))
v3=torch.IntTensor(np.cumsum(np.concatenate(voxel_3, axis=0)))
print(v1.size(),v2.size(),v3.size())

torch.Size([20]) torch.Size([6]) torch.Size([4])


then. into model

In [91]:
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import Optional
from torch import nn


class linear_Attention(nn.Module):
    # flow attention in normal version
    def __init__(self, in_planes):
        super(linear_Attention, self).__init__()
        self.n_heads =8
        self.query_projection = nn.Linear(in_planes, in_planes)
        self.key_projection = nn.Linear(in_planes, in_planes)
        self.value_projection = nn.Linear(in_planes, in_planes)
        self.out_projection = nn.Linear(in_planes, in_planes)
        self.dropout = nn.Dropout(0.05)
        self.eps = 1e-6

    def kernel_method(self, x):
        return torch.sigmoid(x)

    def dot_product(self, q, k, v):
        kv = torch.einsum("hld,hlm->hdm", k, v)
        qkv = torch.einsum("hld,hdm->hlm", q, kv)
        return qkv

    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo   
        PN=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for offset in o:
           x_s=x[PN:int(offset),:]
           x_sub.append(x_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 
           #print(offset,tgt_len, embed_dim)
           PN=int(offset)
        ## input: (L or S) D; output: L D
        ## Note: queries, keys, values are not projected yet
        ## 1. Linear projection
           L=tgt_len
           #print('L',L)
           S=tgt_len
           Q = self.query_projection(x_s).view(L, self.n_heads, -1)
           K = self.key_projection(x_s).view(S, self.n_heads, -1)
           V = self.value_projection(x_s).view(S, self.n_heads, -1)
           #Q = queries.transpose(0, 1)
           #K = keys.transpose(0, 1)
           #V = values.transpose(0, 1)
           # 2. Non-negative projection
           Q = self.kernel_method(Q)
           K = self.kernel_method(K)

           KV = torch.einsum("shd,shm->hmd", K, V)

           # Compute the normalizer
           Z = 1/(torch.einsum("lhd,hd->lh", Q, K.sum(dim=0))+self.eps)

           # Finally compute and return the new values
           V = torch.einsum("lhd,hmd,lh->lhm", Q, KV, Z)

           output = V.reshape(L, -1)
           output = self.out_projection(output)
           output = self.dropout(output)

           out.append(output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out



In [92]:
planes=32
transformer2 = linear_Attention(planes)

right_res = transformer2([coord, feat, v])
right_res1 = transformer2([coord, feat, v1])
right_res2 = transformer2([coord, feat, v2])
right_res3 = transformer2([coord, feat, v3])
print(right_res.shape,right_res1.shape,right_res2.shape,right_res3.shape,)

torch.Size([128, 32]) torch.Size([128, 32]) torch.Size([128, 32]) torch.Size([128, 32])
